# Datasets

## Importing the the relevant libraries

(Note MiniAI is from NBDev functionality, it allows us to export code into libraries that we can 
later use with other notebooks. I still have to install NB Dev and implement this into my code)

In [2]:
from __future__ import annotations
import math,numpy as np,matplotlib.pyplot as plt
from operator import itemgetter
from itertools import zip_longest
import fastcore.all as fc

from torch.utils.data import default_collate

# from miniai.training import *

In [3]:
import logging,pickle,gzip,os,time,shutil,torch,matplotlib as mpl
from pathlib import Path

from torch import tensor,nn,optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from datasets import load_dataset,load_dataset_builder

import torchvision.transforms.functional as TF
# from fastcore.test import test_cl

In [4]:
torch.set_printoptions(precision=2, linewidth=140, sci_mode=False)
torch.manual_seed(1)
mpl.rcParams['image.cmap'] = 'gray'

In [5]:
logging.disable(logging.WARNING)

## Hugging Face datasets

In [6]:
name = 'fashion_mnist'
ds_builder = load_dataset_builder(name)
print(ds_builder.info.description)

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of
60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image,
associated with a label from 10 classes. We intend Fashion-MNIST to serve as a direct drop-in
replacement for the original MNIST dataset for benchmarking machine learning algorithms.
It shares the same image size and structure of training and testing splits.



In [7]:
ds_builder.info.features  

{'image': Image(decode=True, id=None),
 'label': ClassLabel(num_classes=10, names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)}

In [8]:
ds_builder.info.splits

{'train': SplitInfo(name='train', num_bytes=31296655, num_examples=60000, dataset_name='fashion_mnist'),
 'test': SplitInfo(name='test', num_bytes=5233818, num_examples=10000, dataset_name='fashion_mnist')}

In [9]:
# To actually download the dataset
# dsd creates a datasets dictionary

dsd = load_dataset(name)

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset fashion_mnist downloaded and prepared to /root/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
dsd
#Note that this returns dictionaries rather than the tuples we previously used

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [11]:
train, test = dsd['train'], dsd['test']
train[0]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=L size=28x28>,
 'label': 9}

In [12]:
# setting x,y as shorthand for images and labels, so we don't have to type that out everytime we use them
x,y = ds_builder.info.features
x,y

('image', 'label')

In [13]:
img = train[4][x]
img

In [14]:
featy = train.features[y]
featy

ClassLabel(num_classes=10, names=['T - shirt / top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'], id=None)

In [15]:
# Creating a small batch for testing
xb = train[:5][x]
yb = train[:5][y]
yb

[9, 0, 0, 3, 0]

In [16]:
# featy is now a Hugging Face class that allows us to convert the integer values to their actual string descriptions, like so:
featy.int2str(yb)

['Ankle boot',
 'T - shirt / top',
 'T - shirt / top',
 'Dress',
 'T - shirt / top']

### Collating the data so our model can use them

We can define a collate function that returns a dictionary of stacked tensors, 
this was manually created as Pytorch usually collates with tuples, and Hugginface uses dict.
Pytorch is happy with you returning a dictionary for this purpose.

Here we also to all the work of the data aug in the collate function, so as to leave the model less abstracted

In [17]:
def collate_fn(b):
    return {x:torch.stack([TF.to_tensor(o[x]) for o in b]),
            y:tensor([o[y] for o in b])}


In [18]:
dl =DataLoader(train, collate_fn=collate_fn, batch_size=16)
b = next(iter(dl))
b[x].shape, b[y]

(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

We can define a transforms function, where we can then also add data augmentation to the 
model at the level of the dataloader feeding the model. Here we create a transformed dataset,
'tds'

In [19]:
def transforms(b):
    b[x] = [TF.to_tensor(o) for o in b[x]]
    return b

In [20]:
# With this "with_transforms" attribute, transforms is called on the fly, every time __getitem__ is called
# This dataset, otherwise, has an API identical to our previous datasets
# Additionally, we've got a dictionary of tensors, so we don't need a collate function anymore

tds = train.with_transform(transforms)
dl = DataLoader(tds, batch_size=16)
b = next(iter(dl))
b[x].shape, b[y]

(torch.Size([16, 1, 28, 28]),
 tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5, 0, 9, 5, 5, 7, 9]))

So having to return the b seems clunky, aesthetically, so we want to simplify the code,
doing this in place

In [21]:
def _transformi(b): b[x] = [TF.to_tensor(o) for o in b[x]]

In [22]:
# This is a function-generating function that modifies my inplace function to return something

def inplace(f):
    def _f(b):
        f(b)
        return b
    return _f

In [23]:
transformi = inplace(_transformi) 

In [24]:
r = train.with_transform(transformi)[0]
r[x].shape,r[y]  

(torch.Size([1, 28, 28]), 9)

In [25]:
# As this is very common in Python, we can use this decorator
# It takes transformi, sends it to inplace and then replaces it with the result

@inplace
def transformi(b): b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [26]:
tdsf = train.with_transform(transformi)
r = tdsf[0]
r[x].shape,r[y]

(torch.Size([784]), 9)

In [27]:
# Sidenote on some dictionary functionality, we can use itemgetterf
d = dict(a=1,b=2,c=3)
ig = itemgetter('a','c')
ig(d)

(1, 3)

In [28]:
# It doesn't even matter if it is really a dictionary,as long as it looks like one

class D:
    def __getitem__(self, k): return 1 if k=='a' else 2 if k=='b' else 3

In [29]:
d = D()
ig(d)

(1, 3)

In [30]:
list(tdsf.features)

['image', 'label']

Pytorch comes with a default collate function that can take dictionaries, look up the keys in the, 
and collate all the entries matching the keys. This also works on tuples

In [32]:
batch = dict(a=[1],b=[2]), dict(a=[3],b=[4])
default_collate(batch)

{'a': [tensor([1, 3])], 'b': [tensor([2, 4])]}

In [35]:
# So now we can create a function that collates our dictionary data into a tuple, as Hugging Face data is 
# in a dictionary, but Pytorch prefers tuples

def collate_dict(ds):
    get = itemgetter(*ds.features)
    def _f(b): return get(default_collate(b))
    return _f

In [36]:
dlf = DataLoader(tdsf, batch_size=4, collate_fn=collate_dict(tdsf))
xb, yb = next(iter(dlf))
xb.shape, yb

(torch.Size([4, 784]), tensor([9, 0, 0, 3]))

The data now looks like the data we are used to

## Plotting images